# Cloud Native Satellite Data Demo 1
* Use pystac_client to search the STAC API at Planetary Computer
* Use odc.stac to load the data
* Use hvplot to visualize the data

In [ ]:
import pystac_client
import planetary_computer
from rich.table import Table
import hvplot.xarray

# Connect to the Planetary Computer STAC API
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
collections = list(catalog.get_all_collections())
collections.sort(key=lambda c: c.id)
table = Table("ID", "Title", title="Planetary Computer collections")
for collection in collections:
    table.add_row(collection.id, collection.title)
table

In [ ]:
# Define our area of interest (AOI) - a rough box around Cape Cod
bbox = [-71.0, 41.5, -69.8, 42.2]

In [ ]:
# Let's search for Sentinel-2 Level-2A data from this past summer
# with minimal cloud cover.
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime="2025-08-01/2025-09-30",
    query={"eo:cloud_cover": {"lt": 10}}, # less than 10% clouds
)

# Get the results and see what we found
items = search.item_collection()
print(f"Found {len(items)} scenes matching your criteria.")

# Let's inspect the assets of the first item to see the available data bands
if items:
    first_item = items[0]
    print("\nAssets for the first scene:")
    for asset_key, asset in first_item.assets.items():
        print(f"- {asset_key}: {asset.title}")

In [ ]:
# Assuming 'items' is the ItemCollection from the first step
from odc.stac import stac_load


# Load the data using odc-stac.
# Note that we specify the dask chunks as a dictionary.
data_ds = stac_load(
    items,
    bands=["B04", "B03", "B02", "B08"], # B04=Red, B03=Green, B02=Blue, B08=NIR
    bbox=bbox,
    resolution=10,
    chunks={"x": 2048, "y": 2048},
)

# Let's inspect our new xarray Dataset.
# Notice the "Data variables" section.
print(data_ds)

In [ ]:
import hvplot.xarray  # Make sure to import this to activate the .hvplot accessor
import numpy as np

# --- Step 1: Find the clearest scene (same as before) ---
print("Finding the clearest available scene...")
valid_pixels = data_ds.B02.where(data_ds.B02 > 0).count(dim=["x", "y"])
best_time_slice = valid_pixels.argmax().compute()
best_scene = data_ds.isel(time=best_time_slice)
scene_date = np.datetime_as_string(best_scene.time.values, unit='D')
print(f"-> Found scene from: {scene_date}")

# --- Step 2: Prepare the RGB data (same as before) ---
# The order B04, B03, B02 corresponds to Red, Green, Blue
rgb = best_scene[["B04", "B03", "B02"]].to_array(dim="band")

# --- Step 3: Scale the data for visualization (same as before) ---
rgb_scaled = rgb.clip(0, 3000) / 3000

In [ ]:
source_crs = f"{items[0].properties['proj:code']}"
source_crs

In [ ]:
# --- Step 4: Plot with hvplot! ---
print("Generating interactive plot...")
# The .hvplot.rgb method is specifically designed for this task.
# The 'bands' argument tells it which dimension holds the R, G, B channels.
image_plot = rgb_scaled.hvplot.rgb(
    x='x',
    y='y',
    bands='band',
    rasterize=True,    # Essential for good performance with large images
    frame_width=600,
    crs=source_crs,
    geo=True,
    flip_yaxis=False,   # not needed as hvplot figures this out
    title=f"{scene_date}"
)

# In a Jupyter notebook, this will display the interactive plot automatically
image_plot